# Introducción
Esta práctica de la asignatura *Minería de datos* del curso 2023-2024 tiene como objetivo escoger un conjunto de datos y a posteriori, realizar tareas de preprocesado, limpieza de datos, imputación y extracción de características. Además de esto, se deberá realizar un análisis exhaustivo de los mismos, destacando análisis basados en series temporales y medidas de dispersión

Concretamente, nosotros nos hemos centrado en 3 datasets, los cuales son los siguientes:
* *Driver_Licenses_and_ID_Cards_Transferred_to_Washington.csv*: El cual posee la información de los conductores de vehículos en Washington.
* *Washington_State_Cities_and_Counties.csv*: El cual posee los estados pertenecientes a Washington.
* *Electric_Vehicle_Population_Data.csv*: Este posee información acerca de los vehículos eléctricos, como su autonomía o ubicación.

Dados estos conjuntos de datos, se va a proceder con el preprocesado, unión y análisis de los datos.



# Preproceso

## Lectura de los datos

En la siguiente celda, se van a importar las librerías necesarias y leer los datasets que componen nuestros datos:

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Para reproducibilidad de valores aleatorios
np.random.seed(0)

pd.set_option("display.max_columns", None)

df_driv  = pd.read_csv("/kaggle/input/washington-vehicles-data/Driver_Licenses_and_ID_Cards_Transferred_to_Washington.csv")
df_was   = pd.read_csv("/kaggle/input/washington-vehicles-data/Washington_State_Cities_and_Counties.csv")
df_elect = pd.read_csv("/kaggle/input/washington-vehicles-data/Electric_Vehicle_Population_Data.csv")

Una vez leídos los datos, se va a proceder a combinar datos para obtener en un único DataFrame los datos de los vehiculos électricos y las características de las ciudades:

## Datos de vehículos eléctricos

In [ ]:
df = pd.merge(df_elect, df_was, how='left', left_on=['County', 'City'], right_on=['COUNTY NAME', 'CITY NAME'])
df[:3]

Ahora vamos a eliminar las columnas repetidas, originadas al unir los dos DataFrames:

In [ ]:
df = df.drop(columns=['CITY NAME', 'COUNTY NAME'])
df[:3]

Una vez tenemos el dataset listo para su siguiente fase, es decir, el análisis algo más exhaustivo de las columnas necesarias para los posteriores gráficos, vamos a ver las columnas que posee nuestro dataframe y los tipos de datos de cada una de ellas:

In [ ]:
dtypes_df = pd.DataFrame(df.dtypes, columns=['DataType'])

# Reseteamos el índice para convertir los nombres de las columnas en una columna
dtypes_df.reset_index(inplace=True)

# Renombrar las columnas para mayor claridad
dtypes_df.columns = ['Nombre de columna', 'Tipo de dato']
dtypes_df

Donde, como podemos observar, tenemos tanto valores *objects*, que representan cadenas de texto generalmente, *float64*, que representan valores decimales y *int64*, que representan enteros. Destacar que, realmente, la columna 

### Imputación de los datos

En este apartado lo que vamos a hacer es analizar las diferentes columnas por si contienen valores nulos que pueden influir negativamente a la hora de realizar un análisis y resultar en conclusiones negativas.

Como se puede observar a contiuación, existen gran cantidad de valores en la columna de `Electric Range` que son valores nulos, para mitigar los efectos negativos que puede tener a la hora de realizar un análisis, vamos a imputar los valores a partir del valor medio del rango por marca y modelo para aquellos valores que no son cero, lo cual lo podemos observar en la siguiente celda:

In [ ]:
df_electric_range = df[df['Electric Range'] > 0].groupby(['Make'])['Electric Range'].mean().reset_index(drop=False)
df_electric_range[:4]

Ahora, vamos a observar si existe algún registro que tenga el valor de la autonomía del vehículo como valor NaN

In [ ]:
df[df['Electric Range'].isna()]

Donde, como se puede observar, existe un único registro. Este, lo vamos a modificar estableciendo su autonomía a 0, donde, los valores que autonomía 0 serán imputados.

In [ ]:
df['Electric Range']= df['Electric Range'].fillna(0)

Ahora, vamos a definir la función auxiliar que vamos a utilizar para poder imputar los datos, debido a que cada una de las marcas tienen valores distintos de autonomía, se ha decidido establecer el valor de la autonomía de los vehículos que tengan 0 en el campo `Electric Range`, como el valor de la media de la autonomía de los vehículos de la marca que SÍ se tienen registros:

In [ ]:
def obtener_longitud_media(x):
    # En caso de ser mayor que 0, lo dejamos, esto ocurrirá cuando sí se tiene registro
    if float(x['Electric Range']) > 0:
        return x
    else:
        # Obtenemos el valor de la media de la marca del individuo analizado
        range_default =  df_electric_range[(x['Make'] == df_electric_range['Make'])]['Electric Range']
        if len(range_default) > 0:
            # En caso de haber más vehículos con dicha marca, se devuelve su media
            x['Electric Range'] = range_default.iloc[0]
            return x
        else:
            # En caso de no haber registros de dicha marca, se devolverá 0.
            return x

Una vez definida la función, la aplicamos a continuación:

In [ ]:
df = df.apply(obtener_longitud_media,axis=1)

Una vez se ha aplicado la función, vamos a volver a filtrar dentro de nuestro conjunto de datos o DataFrame, las marcas para las cuales no tenemos registros de la autonomía de sus vehículos son las siguientes:

In [ ]:
idx_zero_er = df[df['Electric Range'] == 0]['Make'].index
makes_without_electric_range = df[df['Electric Range'] == 0]['Make'].unique()
makes_without_electric_range 

## Datos de conductores

Ahora se va a tratar con el conjunto de datos de los conductores de vehículos, donde primero, vamos a eliminar las columnas que, a priori, descartamos ya que no nos van a ser de utilidad y convertimos la fecha a tipo *datetime64*:

In [ ]:
df_driv = df_driv.drop(columns=['Year', 'Month Name', 'Month'])
df_driv['Issue Date'] = pd.to_datetime(df_driv['Issue Date'])

Ahora, procederemos de manera similar a lo que hicimos anteriormente con el conjunto de datos de vehículos eléctricos, presentando los tipos de datos correspondientes a cada columna.

In [ ]:
dtypes_df = pd.DataFrame(df_driv.dtypes, columns=['DataType'])

# Reseteamos el índice para convertir los nombres de las columnas en una columna
dtypes_df.reset_index(inplace=True)

# Renombrar las columnas para mayor claridad
dtypes_df.columns = ['Nombre de columna', 'Tipo de dato']
dtypes_df

Donde, como podemos observar, tenemos columnas de tipo int64, datetime64 y object. Destacar que, como en el conjunto de datos anterior, tambien tenemos puntos geográficos (columna *Card Origin*)

In [ ]:
df_driv.sample(3)

# Análisis

## Marcas más comunes

En este apartado vamos a visualizar las marcas de vehículos que predominan en la población de vehículos electrícos/híbridos enchufables en el estado de Washington.

In [ ]:
population_per_maker = df.groupby('Make').size().reset_index(name='Count').sort_values(by= 'Count', ascending=False)
population_per_maker.head(6)

Y ahora vamos a mostrar el top 10 marcas que más vehículos eléctricos han vendido, lo vamos a mostrar a continuación en forma de diagrama de barras:

In [ ]:
fig = px.histogram(population_per_maker.head(10), x='Make',y='Count', title='TOP 10 Marcas de Vehículos  con mayor población de vehículos eléctricos en el Estado de Washington')
fig.update_layout(xaxis_title='Marca', yaxis_title='Número de Vehículos')
fig.show()

Donde, se puede observar un claro dominio de la marca *Tesla* en cuanto a venta de vehículos, vendiendo hasta 7 veces más que la 2º marca. Las siguientes marcas oscilan entre los 5k-12k vehículos vendidos, mientras que Tesla supera con creces los 70k vehículos vendidos. 

Ahora, vamos a mostrar en forma de diagrama de árbol las 20 marcas que más vehículos producen. En este gráfico el total del cuadrado es el 100% de los vehículos eléctricos producidos. Por lo que vamos a ver como se distribuye la producción de vehículos eléctricos por marca:

In [ ]:
import squarify
plt.figure(figsize=(14,10))
df_top_twenty_makers = df['Make'].value_counts()[:20]
squarify.plot(df_top_twenty_makers.values,label=df_top_twenty_makers.index,alpha=0.8)
plt.axis('off')
plt.show()

Donde, de nuevo, destacamos el claro dominio de *Tesla*, seguido de marcas como *Nissan* y *Chevrolet*.

## Modelos más comunes

En este apartado vamos a analizar cuales son los modelos de vehículos más vendidos, donde veremos su marca, modelo y año del vehículo. Primero, vamos a agrupar, calcular el nombre completo de los vehículos y calcular el número de registros que tenemos por cada uno de ellos:

In [ ]:
population_per_model = df.groupby(['Make','Model','Model Year']).size().reset_index(name='Count').sort_values(by= 'Count', ascending=False)
population_per_model['Full Name'] = population_per_model['Make'] +' '+ population_per_model['Model'] + ' '+population_per_model['Model Year'].astype(str)
population_per_model.head(10)

Una vez obtenidos los datos, vamos a mostrarlos en forma de diagrama de barras:

In [ ]:
fig = px.histogram(population_per_model.head(10), x='Full Name',y='Count', title='TOP 10 modelos con mayor población de vehículos eléctricos en el Estado de Washington')
fig.update_layout(xaxis_title='Modelo', yaxis_title='Número de Vehículos')
fig.show()

Donde hay un claro dominio de *Tesla*, ya que los 10 primeros son vehículos de la marca *Tesla*. Destacamos que los más comunes son el Tesla Model 3 y Tesla Model Y, dejando fuera algunos otros modelos de la marca como Model S.

Si nos fijamos en las cifras que aparecen para los modelos de Tesla, estas cifras superan a las ventas totales de ciertas marcas que hemos msotrado en el estudio anterior.

## Producción de Vehículos por Año y Marca

En este apartado, vamos a ver como ha ido evolucionando la producción de los vehículos eléctricos desde el año 1997 hasta la actualidad, por lo que primero vamos a quedarnos únicamente con el años de los vehículos:

In [ ]:
# Obtenemos de todo el conjunto, únicamente los años de lo vehículos
population_model_per_year = df[['Model Year']]
population_model_per_year.head(5)

Y ahora vamos a realizar el histograma de la producción de los vehículos eléctricos por año:

In [ ]:
plt.figure(figsize=(13,7))
sns.histplot(x='Model Year', data=population_model_per_year, bins=[x for x in range(1997,2024)], kde=True)
plt.xlabel("Años")
plt.ylabel("Nº de vehículos eléctricos producidos")

plt.show()

Donde, como podemos observar, ha habido una tendencia creciente desde 2011 hasta la actualidad, experimentando un gran crecimiento en los últimos 3 años.

Ahora vamos a realizar el mismo estudio, pero distinguiendo la marca y el año. Es decir, por cada año, distinguiremos la cantidad de vehículos que produce cada marca. Para realizar esta tarea, primero vamos a agrupar por marca y año:

In [ ]:
population_per_maker_year = df.groupby(['Make','Model Year']).size().reset_index(name='Count').sort_values(by= 'Count', ascending=False)
population_per_maker_year.head(5)

Ahora vamos a obtener las 10 marcas que más producen y vamos a mostrar un subconjunto de los datos, para poder visualizar como se nos queda el DataFrame:

In [ ]:
population_per_maker_top_10 = population_per_maker.head(10)
df_top_10_makers = df[df['Make'].isin(population_per_maker_top_10['Make'])]
df_top_10_makers.head(3)

Y ahora, vamos a volver a realizar el histograma anterior, pero esta vez, distinguiendo por marca:

In [ ]:
plt.figure(figsize=(14, 7))
sns.histplot(x='Model Year', data=df_top_10_makers, bins=[x for x in range(2010,2024)], hue='Make', multiple='stack')
plt.show()

Como se puede observar, hasta 2018 la producción de vehículos estaba más o menos equilibrada. Sin embargo, a partir de 2019, el número de vehículos que produce Tesla empieza a incrementar de forma drástica, dominando por tanto el mercado de vehículos eléctricos desde ese año.

Debido a que Tesla establece un claro dominio, vamos a repetir el mismo histograma, pero esta vez eliminando la marca *Tesla* para poder visualizar de forma más clara como se distribuye la producción de vehículos eléctricos entre las demás marcas:

In [ ]:
population_per_maker_top_10 = population_per_maker.head(10)
df_top_10_makers_no_TESLA = df[df['Make'].isin(population_per_maker_top_10['Make'][1:])]
df_top_10_makers_no_TESLA.head(3)

Una vez hemos eliminado a *Tesla*, mostramos el histograma en cuestión:

In [ ]:
plt.figure(figsize=(14, 7))
sns.histplot(x='Model Year', data=df_top_10_makers_no_TESLA, bins=[x for x in range(2010,2024)], hue='Make', multiple='stack')
plt.show()

Y como se puede observar, la cantidad devehículos está más equilibrada, destacando marcas como *Chevrolet*, *Kia* y *Ford* como las marcas que más vehículos eléctricos producen. 

## Distribución de autonomía por marca

En este apartado, vamos a analizar la autonomía de los vehículos por marca

In [ ]:
df_top_ten_makers = df[df['Make'].isin(population_per_maker.head(10)['Make'])]

In [ ]:
sns.boxplot(df_top_ten_makers[['Make','Electric Range']],x='Make',y='Electric Range')
plt.xticks(rotation=90)
plt.show()

## Análisis de tipo de vehículos eléctricos

En este apartado, mostramos en un diagrama pastel como se distribuye  los tipos de vehículos eléctricos en toda la población de vehículos eléctricos en el Estado de Washington.

En dicha gráfica observamos que los vehículos eléctricos que funcionan con batería `Battery Electric Vehicle (BEV)` constituyen casi un 80% del total de población de vehículos electrificados. Por otra parte observamos con un porcentaje próximo al 22 % los vehículos con tecnología enchufable `Plug-in Hybrid Electric Vehicle (PHEV)`

In [ ]:
conteo_tipo_vehiculos = df['Electric Vehicle Type'].value_counts()
plt.pie(conteo_tipo_vehiculos, labels=conteo_tipo_vehiculos.index, autopct='%1.1f%%')
plt.axis('equal')  # Mantiene el gráfico circular
plt.title('Distribución de tipos de vehículos eléctricos en el estado de Washington')
plt.show()

## Análisis de la autonomía de los vehículos

### Tendencia central

In [ ]:
df_central = df['Electric Range'].drop(idx_zero_er)

In [ ]:
df_mean_er = df.groupby('Make')['Electric Range'].mean().reset_index(name='Mean Electric Range').sort_values(by='Mean Electric Range', ascending = False)
fig = px.histogram(df_mean_er, x='Make', y='Mean Electric Range', title='Histograma de Medias de Autonomía por Marca')

mean = df_central.mean()
median = df_central.median()
mode = df_central.mode()[0]

fig.add_hline(y=mean, line_dash="dot", annotation_text=f'Mean: {mean:.2f}', annotation_position="bottom right")
fig.add_hline(y=median, line_dash="dot", annotation_text=f'Median: {median:.2f}', annotation_position="bottom right")
fig.add_hline(y=mode, line_dash="dot", annotation_text=f'Mode: {mode:.2f}', annotation_position="bottom right")
fig.update_layout(xaxis_title="Marcas", yaxis_title = "Media de autonomía por marca")
fig.show()

En el histograma se puede observar como las marcas de vehículos eléctricos más potentes tienen una mayor autonomía, con una diferencia muy tan grande con el resto de marcas que incluso no llegan a superar el valor medio. 

### Dispersión

In [ ]:
df_dispersion = df.drop(idx_zero_er)
df_dispersion_val = pd.DataFrame()

In [ ]:
# Valor mínimo
df_dispersion_val["Valor mínimo"] = [df_dispersion['Electric Range'].min()]
# Valor máximo
df_dispersion_val["Valor máximo"] = [df_dispersion['Electric Range'].max()]
# Rango de autonomía eléctrica
df_dispersion_val["Rango"] = [df_dispersion['Electric Range'].max() - df_dispersion['Electric Range'].min()]
# Rango intercuartílico
df_dispersion_val['Rango intercuartílico'] = [round(df_dispersion['Electric Range'].quantile(0.75) - df_dispersion['Electric Range'].quantile(0.25),3)]
# Varianza
df_dispersion_val['Varianza'] =[round(df_dispersion['Electric Range'].var(),3)]
# Desviación estándar
df_dispersion_val['Desviación estándar'] =[round(df_dispersion['Electric Range'].std(),3)]
# Coeficiente de variación
df_dispersion_val['Coeficiente de variación'] =[round((df_dispersion['Electric Range'].std() / df_dispersion['Electric Range'].mean()) * 100,3)]

In [ ]:
print(df_dispersion_val.to_string(index = False))

Los valores de dispersión obtenidos son tan grandes debido a la diferencia que existe a la hora de hablar sobre la autonomía de los diferentes vehículos de cada marca. Con la obtención de estos valores grandes, se puede deducir la existencia de datos dispares.

## Mapa de calor de vehículos eléctricos

A continuación vamos a generar un mapa de calor de las ubicaciones de los vehiculos electricos en el estado de Washington. 

Esto lo vamos a realizar mediante la libreria folium.

El primer paso es de crear las columnas 'latitud' y 'longitud' a partir de la columna 'Vehicle Location', una vez creadas estas columnas creamos un nuevo conjunto de datos en el que no aparezcan celdas vacias en las columnas geograficas y lo pasamos a formato lista.

También obtenemos una muestra de los datos, ya que en caso de utilizar todos los datos disponibles los recursos computacionales y el tiempo de ejecución se dispara.

Con todos estos recursos ya se puede crear el HeatMap mediante folium.

In [ ]:
df['longitud'] = df['Vehicle Location'].str.strip('POINT ()').str.split(' ', expand=True)[0]
df['latitud'] = df['Vehicle Location'].str.strip('POINT ()').str.split(' ', expand=True)[1]

df.head()

In [ ]:
import folium
from folium.plugins import HeatMap

#Necesarios los datos sin nan
df_mapa = df.dropna(subset=['longitud', 'latitud'])

#Si el sample es muy grande tarda en ejecutarse
df_mapa = df_mapa.sample(10000)

mapa = folium.Map(location=[47, -120], zoom_start=7)

coordenadas = df_mapa[['latitud', 'longitud']].values.tolist()

mapa_calor = HeatMap(coordenadas)

mapa.add_child(mapa_calor)

mapa

Como se observa en el mapa interactivo creado, se diferencian las ubicaciones o las ciudades en las que hay mayor presencia de vehiculos électricos, asi como las zonas en las que no hay tanta presencia de estos.

También se observan distintos puntos a lo largo del país que informan de la presencia de vehiculos électricos. Creemos que se tratan de errores en el conjunto de datos ya que el dataset debería tener únicamente información sobre el estado de Washington.

## Análisis del dataset de conductores

En este punto vamos a analizar el dataset restante. Se trata de un conjunto de datos que almacena información de los permisos de conducir de personas ajenas al estado de Washington y/o al país.

## Histograma de las expediciones de permisos de conducir en el estado de Washington

En la siguiente porción de código, crearemos un histograma con la cantidad de permisos expedidos por el Estado de Washington, en el cual se observarán la tendencia según el paso del tiempo.

In [ ]:
#Convertir la columna a formato fecha
df_driv['Issue Date'] = pd.to_datetime(df_driv['Issue Date'])

#Creación del gráfico
plt.figure(figsize=(12,6))
plt.hist(df_driv['Issue Date'], bins=30, color='skyblue', edgecolor='black')

plt.title('Histograma de las expediciones')
plt.xlabel('Fecha')
plt.ylabel('Expediciones de permisos de conducir')
plt.xticks(rotation=45)
plt.show()

Como se observa, existe un gran decremento de las expediciones de permisos de conducir a principios de año 2020, evento que relacionamos con la pandemia del Covid-19, pero posteriormente el numero de expediciones vuelve a la normalidad.

En cuanto al resto, la tendencia es de expediciones es similar a lo largo del tiempo, con algunos picos destacables.

## Mapa de calor de los permisos originales expedidos fuera del estado de Washington

El siguiente mapa de calor, creado siguiendo la misma estrategia que el anterior, nos permite analizar las úbicaciones de los permisos de conducir 'antiguos' de las personas, y relacionarlo con la inmigración de las diferentes partes del mundo hacia el estado de Washington.

In [ ]:
df_driv['longitud'] = df_driv['Card Origin'].str.strip('POINT ()').str.split(' ', expand=True)[0]
df_driv['latitud'] = df_driv['Card Origin'].str.strip('POINT ()').str.split(' ', expand=True)[1]

df_driv.head(3)
#Necesarios los datos sin nan
df_mapa = df_driv.dropna(subset=['longitud', 'latitud'])

#Si el sample es muy grande tarda en ejecutarse
df_mapa = df_mapa.sample(10000)

mapa = folium.Map(location=[0, 0], zoom_start=2)

coordenadas = df_mapa[['latitud', 'longitud']].values.tolist()

mapa_calor = HeatMap(coordenadas)

mapa.add_child(mapa_calor)

mapa

En el mapa se pueden observar las tendencias de inmigración hacia el estado de Washington, inmigración procedente de todas partes del mundo, pero sobre todo dentro de los Estados Unidos y Europa Central y del Este.

# Conclusiones

En base a las técnicas de análisis de datos exploratorio estudiadas durante el bloque II de la asignatura de Minería de Datos, hemos seleccionado un conjunto de datos procedente de un portal de datos abiertos que nos permitiese realizar diferentes operaciones de limpieza y análisis. Dicho trabajo se ha llevado a cabo a partir de la colaboración de los diferentes miembros del grupo.

La elección del conjunto de datos se realizó cuidadosamente, optando por datos abiertos del [Portal de datos abiertos de EEUU](https://data.gov/) cuyo ámbito se centraba en la **población de vehículos eléctricos** y **Carnet de Conducir** en el estado de Washington.

Por lo tanto, las tareas que hemos llevado a cabo en este trabajo han sido.

- **Preprocesamiento** : Limpieza, transformación y selección de caracterísitcas para su posterior análisis. 

- **Análisis exploratorio** : Análisis de tendencias centrales y dispersiones para aquellas caracterísitcas numéricas que nos ayudan a comprender la naturaleza de los datos. Además, se complementó con análisis de series temporales para características relacionadas con la población de vehículos eléctricos.

- **Visualización de datos** jugó un papel fundamental en la interpretación y presentación de los resultados. Se emplearon diversas técnicas gráficas para representar los hallazgos de manera clara y comprensible. 

En conclusión, en este cuaderno hemos demostrado la aplicabilidad de las diferentes técnicas aprendidas durante las sesiones y nos proporcionan una base sólida para futuros trabajos.
